# Start working on LLM explanation

In [1]:
import dill
import shap
import pandas as pd
import numpy as np
import os

/home/syeda/.pyenv/versions/scam_job_detector/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "../models/final_model.dill"
preprocessor_path = "../models/preprocessor.dill"
data_path = "../raw_data/data_cleaned.csv"

model_path, preprocessor_path, data_path

('../models/final_model.dill',
 '../models/preprocessor.dill',
 '../raw_data/data_cleaned.csv')

In [3]:
# load the data
df = pd.read_csv(data_path)
df.head(1)

,has_company_logo,employment_type,industry,function,fraudulent,job_description,country
0,1,Other,NaN,Marketing,0,market intern food weve create groundbreaking ...,US


In [4]:
print(df.columns.tolist())

['has_company_logo', 'employment_type', 'industry', 'function', 'fraudulent', 'job_description', 'country']


In [5]:
# load the model and preprocessor
with open(model_path, "rb") as f:
    model = dill.load(f)

with open(preprocessor_path, "rb") as f:
    preprocessor = dill.load(f)

print("Model:", model)
print("Preprocessor loaded:", type(preprocessor))

Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=275, n_jobs=-1,
              num_parallel_tree=None, ...)
Preprocessor loaded: <class 'sklearn.compose._column_transformer.ColumnTransformer'>


# Streamlit Output stimulation

In [6]:
sample = df.sample(1, random_state=42)
sample

,has_company_logo,employment_type,industry,function,fraudulent,job_description,country
4708,1,Full-time,Apparel & Fashion,Information Technology,0,python engineer stylect dynamic startup help h...,GB


In [7]:
# remove target if present
X_sample = sample.drop(columns=["fraudulent"], errors="ignore")

# sanity check
X_sample

,has_company_logo,employment_type,industry,function,job_description,country
4708,1,Full-time,Apparel & Fashion,Information Technology,python engineer stylect dynamic startup help h...,GB


In [8]:
X_transformed = preprocessor.transform(X_sample)

X_transformed.shape

(1, 381412)

In [9]:
prediction = model.predict(X_transformed)[0]
probability = model.predict_proba(X_transformed)[0][1]

label = "FAKE" if prediction == 1 else "REAL"

print(f"Prediction: {label}")
print(f"Fake probability: {probability:.3f}")


Prediction: REAL
Fake probability: 0.000


# SHAP 

In [10]:
type(model)

xgboost.sklearn.XGBClassifier

In [11]:
dir(model)

['_Booster',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_is_fitted__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_request_for_signature',
 '_can_use_inplace_predict',
 '_configure_fit',
 '_create_dmatrix',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_get_default_requests',
 '_get_doc_link',
 '_get_iteration_range',
 '_get_metadata_request',
 '_get_param_names',
 '_get_params_html',
 '_get_type',
 '_html_repr',
 '_load_model_attributes',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_evaluation_result',
 '_update_sklearn_tag

In [12]:
def predict_proba_fn(X):
    return model.predict_proba(X)[:, 1]

In [13]:
background_df = (
    df
    .drop(columns=["fraudulent"], errors="ignore")
    .sample(20, random_state=42)
)

X_background = preprocessor.transform(background_df)


In [14]:
X_background.shape

(20, 381412)

In [15]:
X_transformed.shape

(1, 381412)

In [16]:
explainer = shap.KernelExplainer(
    predict_proba_fn,
    X_background
)


In [17]:
shap_values = explainer.shap_values(X_transformed)

  0%|          | 0/1 [00:00<?, ?it/s]/home/syeda/.pyenv/versions/scam_job_detector/lib/python3.10/site-packages/xgboost/data.py:1281: UserWarning: Unknown data type: <class 'scipy.sparse._lil.lil_matrix'>, trying to convert it to csr_matrix
  warnings.warn(
100%|██████████| 1/1 [03:00<00:00, 180.74s/it]


In [18]:
shap_values.shape

(1, 381412)

In [19]:
feature_names = preprocessor.get_feature_names_out()
len(feature_names), shap_values.shape[1]


(381412, 381412)

In [20]:
shap_df = pd.DataFrame({
    "feature": feature_names,
    "shap_value": shap_values[0]
})

shap_df["abs_value"] = shap_df["shap_value"].abs()

top_features = shap_df.sort_values(
    "abs_value", ascending=False
).head(10)

In [21]:
top_features[["feature", "shap_value", "abs_value"]]

,feature,shap_value,abs_value
263,pipeline-2__has_company_logo_0,-0.007925,0.007925
83,pipeline-1__country_US,-0.005236,0.005236
352159,columntransformer__tfidfvectorizer__understand,-0.005082,0.005082
138686,columntransformer__tfidfvectorizer__fun,-0.004935,0.004935
108334,columntransformer__tfidfvectorizer__engineer,-0.004718,0.004718
354727,columntransformer__tfidfvectorizer__urgent,-0.004548,0.004548
371119,columntransformer__tfidfvectorizer__work,-0.004185,0.004185
193418,columntransformer__tfidfvectorizer__love,-0.003648,0.003648
330369,columntransformer__tfidfvectorizer__systems,-0.003577,0.003577
85563,columntransformer__tfidfvectorizer__department,-0.002959,0.002959


In [22]:
for _, row in top_features.iterrows():
    direction = "FAKE ↑" if row["shap_value"] > 0 else "REAL ↓"
    print(f"{row['feature']}: {direction} ({row['shap_value']:.4f})")


pipeline-2__has_company_logo_0: REAL ↓ (-0.0079)
pipeline-1__country_US: REAL ↓ (-0.0052)
columntransformer__tfidfvectorizer__understand: REAL ↓ (-0.0051)
columntransformer__tfidfvectorizer__fun: REAL ↓ (-0.0049)
columntransformer__tfidfvectorizer__engineer: REAL ↓ (-0.0047)
columntransformer__tfidfvectorizer__urgent: REAL ↓ (-0.0045)
columntransformer__tfidfvectorizer__work: REAL ↓ (-0.0042)
columntransformer__tfidfvectorizer__love: REAL ↓ (-0.0036)
columntransformer__tfidfvectorizer__systems: REAL ↓ (-0.0036)
columntransformer__tfidfvectorizer__department: REAL ↓ (-0.0030)


In [25]:
def clean_feature_name(feature: str) -> str:
    # TF-IDF word features
    if "tfidfvectorizer__" in feature:
        return f"word '{feature.split('__')[-1]}' in job description"

    # binary categorical features
    if "has_company_logo" in feature:
        return "company has a logo"

    if "country_" in feature:
        return f"job country is {feature.split('_')[-1]}"

    # fallback
    return feature


In [26]:
top_features["clean_feature"] = top_features["feature"].apply(clean_feature_name)

In [ ]:
explanation_payload = {
    "model_prediction": {},
    "top_features": []
}

# model-level prediction
explanation_payload["model_prediction"] = {
    "label": "FAKE" if prediction == 1 else "REAL",
    "fake_probability": float(probability)
}

# feature-level explanations (SHAP)
for _, row in top_features.iterrows():
    explanation_payload["top_features"].append({
        "feature": row["clean_feature"],
        "shap_value": float(row["shap_value"]),
        "direction": "FAKE" if row["shap_value"] > 0 else "REAL"
    })


In [29]:
from pprint import pprint
pprint(explanation_payload)

{'model_prediction': {'fake_probability': 6.67306812829338e-06,
                      'label': 'REAL'},
 'top_features': [{'direction': 'REAL',
                   'feature': 'company has a logo',
                   'shap_value': -0.007924503029061163},
                  {'direction': 'REAL',
                   'feature': 'job country is US',
                   'shap_value': -0.005236243055372775},
                  {'direction': 'REAL',
                   'feature': "word 'understand' in job description",
                   'shap_value': -0.005081587556706425},
                  {'direction': 'REAL',
                   'feature': "word 'fun' in job description",
                   'shap_value': -0.004935120473308591},
                  {'direction': 'REAL',
                   'feature': "word 'engineer' in job description",
                   'shap_value': -0.004718441597824279},
                  {'direction': 'REAL',
                   'feature': "word 'urgent' in job description",
 

In [ ]:
import os
from openai import OpenAI
import json
print("import ok")

import ok


In [40]:
system_prompt = """
You explain machine learning predictions to non-technical users.

Rules:
- Use ONLY the provided information
- Do NOT invent features
- Do NOT change the prediction
- Keep it short (max 3 sentences)
"""

user_prompt = f"""
Explain the following model output in simple English.

JSON:
{json.dumps(explanation_payload, indent=2)}
"""


In [44]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.2
)

print(response.output_text)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}